# 056: RNN, LSTM, GRU## 📚 Learning ObjectivesBy the end of this notebook, you will master:1. **Sequential Data Processing** - Understand why CNNs fail on temporal sequences2. **RNN Architecture** - Hidden states, recurrent connections, backpropagation through time3. **Vanishing Gradient Problem** - Why vanilla RNNs fail on long sequences4. **LSTM Networks** - Cell states, gates (forget, input, output), gradient flow5. **GRU Networks** - Simplified gating (reset, update), computational efficiency6. **Bidirectional RNNs** - Process sequences forward and backward simultaneously7. **Semiconductor Applications** - Sequential test pattern analysis, time-series yield prediction8. **Production Deployment** - Model optimization, inference strategies, real-time processing---## 🎯 Why Sequential Models Matter### **The Temporal Challenge****CNNs assume spatial independence:**- Each pixel processed independently (with local context via filters)- No concept of "before" or "after"- Works for images (spatial data) but fails for sequences (temporal data)**Sequential data requires memory:**- Test results at time $t$ depend on previous measurements at $t-1, t-2, ...$- Wafer yield patterns evolve over production days- Device parametric drift accumulates over time**Example: Test Pattern Analysis**```Parametric Test Sequence (20 measurements over time):Time:    t1    t2    t3    t4    t5    ...  t20Vdd:    1.05  1.06  1.04  1.03  1.02  ...  0.98  ← Voltage drift detectedIdd:    250   252   248   245   240   ...  210   ← Current decreasingFreq:   2.4   2.4   2.3   2.3   2.2   ...  2.0   ← Frequency degradationPattern: Gradual performance degradation → Predict failure at t25```**CNN would analyze each timestep independently (wrong!)**  **RNN captures temporal dependencies (correct!)**---## 🏭 Semiconductor Use Case: Sequential Test Analysis### **Problem Statement****Objective:** Predict device failure 5 test cycles ahead based on parametric trend analysis**Business Value:**- **Proactive binning:** $20M-$80M/year from early failure detection- **Test time reduction:** Skip remaining tests if failure predicted (30% time savings)- **Yield optimization:** Identify slow degradation patterns for process tuning**Data:**- 50,000 devices tested over 20 cycles (1 week of production)- 15 parametric measurements per cycle (Vdd, Idd, freq, power, temp, etc.)- Sequential pattern: Each device has time-series: $(x_1, x_2, ..., x_{20}) \rightarrow y_{failure}$**Challenge:**- Long sequences (20 timesteps)- Multiple features (15 parameters)- Temporal dependencies (drift accumulates over time)- Need to predict 5 cycles ahead (early warning)---## 📊 What We'll Build```mermaidgraph LR    A[Sequential Test Data<br/>20 cycles × 15 params] --> B[Data Preprocessing<br/>Normalization + Windowing]    B --> C[Vanilla RNN<br/>Baseline Model]    B --> D[LSTM Network<br/>Long-term Dependencies]    B --> E[GRU Network<br/>Efficient Alternative]    B --> F[Bidirectional LSTM<br/>Forward + Backward]        C --> G[Comparison<br/>Accuracy + Speed]    D --> G    E --> G    F --> G        G --> H[Best Model Selection<br/>Deploy to Production]        H --> I[Real-time Inference<br/>Predict at Cycle 15]        style A fill:#e1f5ff    style H fill:#d4edda    style I fill:#fff3cd```**Architecture Comparison:**1. **Vanilla RNN:** Simple recurrence, fails on long sequences (vanishing gradient)2. **LSTM:** Gated architecture, excellent long-term memory, 4× parameters of RNN3. **GRU:** Simplified gates, 75% of LSTM parameters, competitive accuracy4. **Bidirectional LSTM:** Process both directions, best accuracy, 2× inference time---## 🔧 Prerequisites```python# Core librariesimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as sns# PyTorch for neural networksimport torchimport torch.nn as nnimport torch.optim as optimfrom torch.utils.data import Dataset, DataLoader, TensorDataset# Scikit-learn utilitiesfrom sklearn.preprocessing import StandardScalerfrom sklearn.model_selection import train_test_splitfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix# Visualizationimport warningswarnings.filterwarnings('ignore')# Set random seeds for reproducibilitynp.random.seed(42)torch.manual_seed(42)```**Installation:**```bashpip install torch numpy pandas matplotlib seaborn scikit-learn```---## 📈 Success Metrics**Model Performance:**- **Accuracy:** ≥85% on failure prediction (5 cycles ahead)- **Recall:** ≥90% (critical: don't miss failures)- **Precision:** ≥80% (minimize false alarms)- **Early detection:** Predict failure at cycle 15 (5 cycles early)**Computational Efficiency:**- **Training time:** <10 min on CPU for 50K devices- **Inference time:** <1ms per device (real-time capable)- **Model size:** <5MB (edge deployable)**Business Impact:**- **Cost savings:** $20M-$80M/year from early detection- **Test time reduction:** 30% (skip remaining tests)- **False alarm rate:** <10% (avoid unnecessary quarantine)---## 🗂️ Notebook Structure1. **Mathematical Foundations** - RNN equations, LSTM gates, GRU mechanics2. **Data Generation** - Synthetic sequential test data with realistic patterns3. **Vanilla RNN Implementation** - From scratch + PyTorch comparison4. **LSTM Network** - Gated recurrence for long sequences5. **GRU Network** - Efficient alternative to LSTM6. **Bidirectional LSTM** - Forward + backward processing7. **Model Comparison** - Accuracy, speed, memory analysis8. **Real-World Projects** - 8 production-ready applications9. **Key Takeaways** - When to use each architecture, best practicesLet's start! 🚀

# 📐 Part 1: Mathematical Foundations

## 🔄 Vanilla RNN Architecture

### **Core Concept: Hidden State as Memory**

**Feedforward networks (CNNs, MLPs):** No memory between inputs  
**Recurrent networks:** Hidden state $h_t$ carries information from previous timesteps

### **RNN Equations**

At each timestep $t$, given input $x_t$ and previous hidden state $h_{t-1}$:

$$
h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)
$$

$$
y_t = W_{hy} h_t + b_y
$$

**Where:**
- $x_t \in \mathbb{R}^{d}$ : Input at time $t$ (e.g., 15 parametric measurements)
- $h_t \in \mathbb{R}^{h}$ : Hidden state at time $t$ (e.g., 128 units)
- $y_t \in \mathbb{R}^{c}$ : Output at time $t$ (e.g., 2 classes: pass/fail)
- $W_{xh} \in \mathbb{R}^{h \times d}$ : Input-to-hidden weights
- $W_{hh} \in \mathbb{R}^{h \times h}$ : Hidden-to-hidden (recurrent) weights
- $W_{hy} \in \mathbb{R}^{c \times h}$ : Hidden-to-output weights
- $b_h, b_y$ : Bias terms
- $\tanh$ : Activation function (outputs in range $[-1, 1]$)

### **Unrolled RNN Visualization**

```
Input sequence:  x₁    x₂    x₃    ...   x₂₀
                  ↓     ↓     ↓           ↓
Hidden states:   h₁ → h₂ → h₃ → ... → h₂₀
                  ↓     ↓     ↓           ↓
Outputs:         y₁    y₂    y₃    ...   y₂₀

At timestep t=3:
  h₃ = tanh(W_hh·h₂ + W_xh·x₃ + b_h)
  y₃ = W_hy·h₃ + b_y

h₃ contains information from x₁, x₂, x₃ (memory!)
```

### **Example Calculation**

**Setup:**
- Input dimension $d=15$ (15 parametric measurements)
- Hidden dimension $h=128$
- Output dimension $c=2$ (binary classification: pass/fail)

**Timestep t=1:**

$$
h_1 = \tanh(W_{hh} h_0 + W_{xh} x_1 + b_h)
$$

where $h_0 = \mathbf{0}$ (initial hidden state is zeros)

$$
h_1 = \tanh\left(\begin{bmatrix} 128 \times 128 \end{bmatrix} \begin{bmatrix} 0 \\ \vdots \\ 0 \end{bmatrix} + \begin{bmatrix} 128 \times 15 \end{bmatrix} \begin{bmatrix} 1.05 \\ 250 \\ \vdots \\ 2.4 \end{bmatrix} + \begin{bmatrix} 128 \times 1 \end{bmatrix}\right)
$$

Result: $h_1 \in \mathbb{R}^{128}$ (values in range $[-1, 1]$)

**Timestep t=2:**

$$
h_2 = \tanh(W_{hh} h_1 + W_{xh} x_2 + b_h)
$$

Now $h_1$ from previous step is used! $h_2$ contains information from both $x_1$ and $x_2$.

**Final output (at t=20):**

$$
y_{20} = W_{hy} h_{20} + b_y = \begin{bmatrix} 2 \times 128 \end{bmatrix} \begin{bmatrix} h_{20} \end{bmatrix} + \begin{bmatrix} 2 \times 1 \end{bmatrix}
$$

$$
y_{20} = \begin{bmatrix} \text{logit}_{\text{pass}} \\ \text{logit}_{\text{fail}} \end{bmatrix} \rightarrow \text{softmax} \rightarrow \begin{bmatrix} P(\text{pass}) \\ P(\text{fail}) \end{bmatrix}
$$

---

## ⚠️ The Vanishing Gradient Problem

### **Backpropagation Through Time (BPTT)**

Training RNNs requires computing gradients with respect to $W_{hh}$ across all timesteps:

$$
\frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^{T} \frac{\partial L_t}{\partial W_{hh}}
$$

For each timestep $t$, gradient flows backward through time:

$$
\frac{\partial L_t}{\partial W_{hh}} = \frac{\partial L_t}{\partial h_t} \cdot \frac{\partial h_t}{\partial h_{t-1}} \cdot \frac{\partial h_{t-1}}{\partial h_{t-2}} \cdots \frac{\partial h_1}{\partial W_{hh}}
$$

### **Chain Rule Explosion**

Each $\frac{\partial h_t}{\partial h_{t-1}}$ term involves:

$$
\frac{\partial h_t}{\partial h_{t-1}} = W_{hh}^T \cdot \text{diag}(\tanh'(h_t))
$$

For long sequences ($T=20$), gradient becomes:

$$
\frac{\partial L_{20}}{\partial W_{hh}} \propto \left(W_{hh}^T\right)^{20} \cdot \prod_{t=1}^{20} \tanh'(h_t)
$$

**Problem 1: $\tanh'(x) \leq 1$** (derivative saturates)

$$
\tanh'(x) = 1 - \tanh^2(x) \leq 1
$$

For $|x| > 2$: $\tanh'(x) \approx 0$ (gradient vanishes)

**Problem 2: Matrix power $(W_{hh}^T)^{20}$**

If eigenvalues of $W_{hh}$ are:
- $\lambda < 1$ : $(W_{hh})^{20} \rightarrow 0$ (vanishing gradient)
- $\lambda > 1$ : $(W_{hh})^{20} \rightarrow \infty$ (exploding gradient)

### **Numerical Example**

Assume $W_{hh}$ has largest eigenvalue $\lambda = 0.9$:

$$
\text{Gradient contribution from timestep } t=1 \text{ to } t=20:
$$

$$
(0.9)^{20} \approx 0.12 \quad \text{(12% of original gradient)}
$$

If $\lambda = 0.8$:

$$
(0.8)^{20} \approx 0.01 \quad \text{(1% survives!)}
$$

**Consequence:** Early timesteps ($t=1, 2, 3$) contribute negligibly to gradient → RNN forgets long-term dependencies!

---

## 🛡️ LSTM: Long Short-Term Memory

### **Key Innovation: Cell State $C_t$ as Protected Memory**

LSTM adds a **cell state** $C_t$ that flows through time with minimal modifications:

$$
C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t
$$

**Why this solves vanishing gradient:**
- Gradient flows directly through $C_t$ without multiplicative $W_{hh}$ terms
- Gates $f_t, i_t$ control information flow (learned, not fixed)
- If $f_t \approx 1$: $C_t \approx C_{t-1}$ (gradient flows unchanged!)

### **LSTM Equations (Complete Architecture)**

At each timestep $t$:

**1. Forget Gate** (decide what to forget from $C_{t-1}$):

$$
f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
$$

- $f_t \in [0, 1]^h$ : Elementwise forget weights
- $f_t = 1$ : Keep everything from $C_{t-1}$
- $f_t = 0$ : Completely forget $C_{t-1}$

**2. Input Gate** (decide what new information to add):

$$
i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
$$

$$
\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)
$$

- $i_t \in [0, 1]^h$ : Input gate weights
- $\tilde{C}_t \in [-1, 1]^h$ : Candidate cell state

**3. Cell State Update** (combine forget and input):

$$
C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t
$$

**4. Output Gate** (decide what to output from $C_t$):

$$
o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
$$

$$
h_t = o_t \odot \tanh(C_t)
$$

**5. Final Output:**

$$
y_t = W_y h_t + b_y
$$

### **LSTM Dimensions**

For our semiconductor example:
- Input: $x_t \in \mathbb{R}^{15}$ (15 parameters)
- Hidden state: $h_t \in \mathbb{R}^{128}$
- Cell state: $C_t \in \mathbb{R}^{128}$
- Concatenated input: $[h_{t-1}, x_t] \in \mathbb{R}^{143}$

Weight matrices:
- $W_f, W_i, W_o, W_C \in \mathbb{R}^{128 \times 143}$
- Total parameters: $4 \times (128 \times 143) = 73,216$ (per LSTM layer)

**4× more parameters than vanilla RNN** (which has only $W_{hh} \in \mathbb{R}^{128 \times 128}$)

### **Why LSTM Works: Gradient Flow Analysis**

Gradient of loss with respect to $C_{t-1}$:

$$
\frac{\partial L}{\partial C_{t-1}} = \frac{\partial L}{\partial C_t} \cdot \frac{\partial C_t}{\partial C_{t-1}} = \frac{\partial L}{\partial C_t} \cdot f_t
$$

**Key insight:** $f_t$ is learned (not fixed)!
- If long-term dependency needed: $f_t \approx 1$ (gradient flows unchanged)
- If short-term needed: $f_t \approx 0$ (gradient blocked)

Compare to vanilla RNN:

$$
\frac{\partial h_t}{\partial h_{t-1}} = W_{hh}^T \cdot \tanh'(h_t)
$$

Fixed matrix $W_{hh}$ (not adaptive!) → Cannot learn to preserve gradients selectively.

---

## ⚡ GRU: Gated Recurrent Unit

### **Simplification of LSTM**

GRU combines forget and input gates into a single **update gate** $z_t$:
- No separate cell state $C_t$ (uses $h_t$ directly)
- Fewer parameters (3 gates vs 4 in LSTM)
- Faster training and inference

### **GRU Equations**

**1. Update Gate** (how much to update hidden state):

$$
z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z)
$$

**2. Reset Gate** (how much past to forget when computing candidate):

$$
r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r)
$$

**3. Candidate Hidden State:**

$$
\tilde{h}_t = \tanh(W_h \cdot [r_t \odot h_{t-1}, x_t] + b_h)
$$

**4. Hidden State Update:**

$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
$$

**Interpretation:**
- $z_t \approx 0$ : Keep old hidden state $h_{t-1}$ (like LSTM forget gate $f_t=1$)
- $z_t \approx 1$ : Use new candidate $\tilde{h}_t$ (like LSTM input gate $i_t=1$)
- $r_t \approx 0$ : Ignore past when computing $\tilde{h}_t$ (fresh start)
- $r_t \approx 1$ : Use past fully (retain memory)

### **GRU vs LSTM: Parameter Comparison**

For hidden size $h=128$, input size $d=15$:

**LSTM:**
- 4 gates: $f_t, i_t, o_t, \tilde{C}_t$
- Parameters: $4 \times (128 \times 143) = 73,216$

**GRU:**
- 3 gates: $z_t, r_t, \tilde{h}_t$
- Parameters: $3 \times (128 \times 143) = 54,912$

**GRU has 75% of LSTM parameters** (25% reduction!)

---

## 🔄 Bidirectional RNNs

### **Forward + Backward Processing**

Standard RNN processes sequence left-to-right: $x_1 \rightarrow x_2 \rightarrow \cdots \rightarrow x_{20}$

**Bidirectional RNN (BiRNN):** Process both directions simultaneously

$$
\overrightarrow{h}_t = \text{RNN}_{\text{forward}}(x_t, \overrightarrow{h}_{t-1})
$$

$$
\overleftarrow{h}_t = \text{RNN}_{\text{backward}}(x_t, \overleftarrow{h}_{t+1})
$$

$$
h_t = [\overrightarrow{h}_t; \overleftarrow{h}_t] \quad \text{(concatenate)}
$$

### **Why Bidirectional Helps**

**Example:** Predict failure at $t=15$ based on full sequence $x_1, ..., x_{20}$

Standard LSTM at $t=15$:
- Sees $x_1, ..., x_{15}$ (forward context)
- Misses $x_{16}, ..., x_{20}$ (future context)

Bidirectional LSTM at $t=15$:
- $\overrightarrow{h}_{15}$ : Information from $x_1, ..., x_{15}$
- $\overleftarrow{h}_{15}$ : Information from $x_{20}, ..., x_{16}$
- Combined $h_{15}$ : Full sequence context!

**Trade-off:** 2× parameters, 2× inference time (but higher accuracy)

---

## 📊 Architecture Comparison Summary

| Architecture | Parameters | Gradient Flow | Long-term Memory | Speed | Use Case |
|--------------|-----------|---------------|------------------|-------|----------|
| **Vanilla RNN** | $h^2 + h \cdot d$ | Poor (vanishing) | ✗ (5-10 steps) | Fast | Short sequences |
| **LSTM** | $4(h^2 + h \cdot d)$ | Excellent | ✓ (100+ steps) | Slow | Long sequences |
| **GRU** | $3(h^2 + h \cdot d)$ | Very good | ✓ (50+ steps) | Medium | Balanced |
| **Bidirectional LSTM** | $2 \times 4(h^2 + h \cdot d)$ | Excellent | ✓ (100+ steps) | Very slow | Best accuracy |

**For $h=128, d=15$:**
- Vanilla RNN: ~18K parameters
- LSTM: ~73K parameters
- GRU: ~55K parameters
- BiLSTM: ~146K parameters

---

## 🎯 Next Steps

Now that we understand the theory, let's implement:
1. Generate synthetic sequential test data
2. Build all 4 architectures from scratch
3. Compare performance on failure prediction task
4. Deploy best model to production

Let's code! 🚀

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
# ========================================
# PART 2: DATA GENERATION
# Sequential Parametric Test Data with Realistic Patterns
# ========================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
np.random.seed(42)
print("=" * 60)
print("SEQUENTIAL TEST DATA GENERATION")
print("=" * 60)
# ========================================
# CONFIGURATION
# ========================================
NUM_DEVICES = 50000      # Total devices to simulate
SEQ_LENGTH = 20          # Test cycles per device
NUM_FEATURES = 15        # Parametric measurements per cycle
FAILURE_RATE = 0.20      # 20% of devices will fail
# Feature names (semiconductor parametric tests)
FEATURE_NAMES = [
    'Vdd_voltage',        # Supply voltage (V)
    'Idd_current',        # Supply current (mA)
    'Frequency',          # Operating frequency (GHz)
    'Power',              # Power consumption (W)
    'Temperature',        # Junction temperature (°C)
    'Leakage_current',    # Standby current (µA)
    'Rise_time',          # Signal rise time (ps)
    'Fall_time',          # Signal fall time (ps)
    'Setup_time',         # Setup time margin (ps)
    'Hold_time',          # Hold time margin (ps)
    'Jitter',             # Clock jitter (ps)
    'Phase_noise',        # PLL phase noise (dBc/Hz)
    'THD',                # Total harmonic distortion (%)
    'SNR',                # Signal-to-noise ratio (dB)
    'BER'                 # Bit error rate (log scale)
]
print(f"\nDataset Configuration:")
print(f"  Devices:          {NUM_DEVICES:,}")
print(f"  Sequence length:  {SEQ_LENGTH} cycles")
print(f"  Features:         {NUM_FEATURES}")
print(f"  Failure rate:     {FAILURE_RATE*100:.0f}%")
# ========================================
# GENERATE SEQUENTIAL DATA
# ========================================
def generate_device_sequence(device_id, will_fail):
    """
    Generate sequential test data for one device
    
    Patterns:
    - Passing devices: Stable parameters with minor noise
    - Failing devices: Gradual drift + accelerated degradation in last 5 cycles
    """
    
    # Base values (nominal operating conditions)
    base_values = np.array([
        1.05,   # Vdd (V)
        250,    # Idd (mA)
        2.4,    # Freq (GHz)
        0.6,    # Power (W)
        75,     # Temp (°C)
        10,     # Leakage (µA)
        50,     # Rise time (ps)
        50,     # Fall time (ps)
        100,    # Setup time (ps)
        100,    # Hold time (ps)
        20,     # Jitter (ps)
        -80,    # Phase noise (dBc/Hz)
        1.0,    # THD (%)
        40,     # SNR (dB)
        -9      # BER (log10)
    ])
    
    sequence = np.zeros((SEQ_LENGTH, NUM_FEATURES))
    
    if will_fail:
        # Failing device: Gradual degradation
        for t in range(SEQ_LENGTH):
            # Drift rate increases over time
            drift_factor = (t / SEQ_LENGTH) ** 2  # Quadratic acceleration
            
            # Different parameters degrade differently
            drift = np.array([
                -0.01 * drift_factor,   # Vdd decreases
                -2.0 * drift_factor,    # Idd decreases (less current)
                -0.05 * drift_factor,   # Frequency drops
                -0.02 * drift_factor,   # Power decreases
                +2.0 * drift_factor,    # Temperature increases (bad!)
                +1.5 * drift_factor,    # Leakage increases (bad!)
                +3.0 * drift_factor,    # Rise time increases (slower)
                +3.0 * drift_factor,    # Fall time increases (slower)
                -5.0 * drift_factor,    # Setup time decreases (margin loss)
                -5.0 * drift_factor,    # Hold time decreases (margin loss)
                +2.0 * drift_factor,    # Jitter increases (bad!)
                +5.0 * drift_factor,    # Phase noise increases (worse)
                +0.5 * drift_factor,    # THD increases (distortion)
                -2.0 * drift_factor,    # SNR decreases (noise)
                +0.5 * drift_factor     # BER increases (errors)
            ])
            
            # Add random noise
            noise = np.random.normal(0, 0.02, NUM_FEATURES)
            
            sequence[t] = base_values + drift + noise
    
    else:
        # Passing device: Stable with minor variations
        for t in range(SEQ_LENGTH):
            # Small random walk
            noise = np.random.normal(0, 0.01, NUM_FEATURES)
            sequence[t] = base_values + noise
    
    return sequence
# Generate dataset
print("\nGenerating sequential test data...")
X = np.zeros((NUM_DEVICES, SEQ_LENGTH, NUM_FEATURES))
y = np.zeros(NUM_DEVICES, dtype=int)
num_fail = int(NUM_DEVICES * FAILURE_RATE)
num_pass = NUM_DEVICES - num_fail
# Generate failing devices
for i in range(num_fail):
    X[i] = generate_device_sequence(i, will_fail=True)
    y[i] = 1  # Failure
# Generate passing devices
for i in range(num_fail, NUM_DEVICES):
    X[i] = generate_device_sequence(i, will_fail=False)
    y[i] = 0  # Pass
print(f"  Passing devices:  {num_pass:,} ({num_pass/NUM_DEVICES*100:.1f}%)")
print(f"  Failing devices:  {num_fail:,} ({num_fail/NUM_DEVICES*100:.1f}%)")


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# VISUALIZATION: EXAMPLE SEQUENCES
# ========================================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle("Sequential Test Patterns: Passing vs Failing Devices", fontsize=16, fontweight='bold')
# Select 3 features to visualize
features_to_plot = [0, 1, 4]  # Vdd, Idd, Temperature
feature_labels = ['Vdd (V)', 'Idd (mA)', 'Temperature (°C)']
for col, (feat_idx, feat_label) in enumerate(zip(features_to_plot, feature_labels)):
    # Passing device (row 0)
    ax_pass = axes[0, col]
    pass_device_idx = num_fail + 0  # First passing device
    ax_pass.plot(range(SEQ_LENGTH), X[pass_device_idx, :, feat_idx], 
                marker='o', linewidth=2, color='green', alpha=0.7)
    ax_pass.set_title(f"Passing Device: {feat_label}", fontweight='bold')
    ax_pass.set_xlabel("Test Cycle")
    ax_pass.set_ylabel(feat_label)
    ax_pass.grid(True, alpha=0.3)
    ax_pass.axhline(y=X[pass_device_idx, 0, feat_idx], color='blue', 
                   linestyle='--', alpha=0.5, label='Baseline')
    ax_pass.legend()
    
    # Failing device (row 1)
    ax_fail = axes[1, col]
    fail_device_idx = 0  # First failing device
    ax_fail.plot(range(SEQ_LENGTH), X[fail_device_idx, :, feat_idx], 
                marker='o', linewidth=2, color='red', alpha=0.7)
    ax_fail.set_title(f"Failing Device: {feat_label}", fontweight='bold')
    ax_fail.set_xlabel("Test Cycle")
    ax_fail.set_ylabel(feat_label)
    ax_fail.grid(True, alpha=0.3)
    ax_fail.axhline(y=X[fail_device_idx, 0, feat_idx], color='blue', 
                   linestyle='--', alpha=0.5, label='Baseline')
    ax_fail.legend()
plt.tight_layout()
plt.savefig('sequential_test_patterns.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Visualization saved to sequential_test_patterns.png")
# ========================================
# FEATURE SCALING
# ========================================
print("\nApplying feature scaling...")
# Reshape for scaling: (num_devices * seq_length, num_features)
X_reshaped = X.reshape(-1, NUM_FEATURES)
scaler = StandardScaler()
X_scaled_reshaped = scaler.fit_transform(X_reshaped)
# Reshape back: (num_devices, seq_length, num_features)
X_scaled = X_scaled_reshaped.reshape(NUM_DEVICES, SEQ_LENGTH, NUM_FEATURES)
print(f"  Original shape:  {X.shape}")
print(f"  Scaled shape:    {X_scaled.shape}")
# Check scaling
print(f"\n  Mean (should be ~0):  {X_scaled_reshaped.mean(axis=0)[:3]}")
print(f"  Std (should be ~1):   {X_scaled_reshaped.std(axis=0)[:3]}")
# ========================================
# TRAIN/VAL/TEST SPLIT
# ========================================
print("\nSplitting dataset...")
# First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y
)
# Second split: 80% train, 20% val (of the 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.20, random_state=42, stratify=y_temp
)
print(f"  Train set:  {X_train.shape[0]:,} devices ({X_train.shape[0]/NUM_DEVICES*100:.1f}%)")
print(f"    Pass:     {(y_train == 0).sum():,}")
print(f"    Fail:     {(y_train == 1).sum():,}")
print(f"\n  Val set:    {X_val.shape[0]:,} devices ({X_val.shape[0]/NUM_DEVICES*100:.1f}%)")
print(f"    Pass:     {(y_val == 0).sum():,}")
print(f"    Fail:     {(y_val == 1).sum():,}")
print(f"\n  Test set:   {X_test.shape[0]:,} devices ({X_test.shape[0]/NUM_DEVICES*100:.1f}%)")
print(f"    Pass:     {(y_test == 0).sum():,}")
print(f"    Fail:     {(y_test == 1).sum():,}")


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# CONVERT TO PYTORCH TENSORS
# ========================================
import torch
from torch.utils.data import TensorDataset, DataLoader
# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)
# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Create data loaders
BATCH_SIZE = 128
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
print(f"\nData loaders created:")
print(f"  Batch size:      {BATCH_SIZE}")
print(f"  Train batches:   {len(train_loader)}")
print(f"  Val batches:     {len(val_loader)}")
print(f"  Test batches:    {len(test_loader)}")
# ========================================
# EARLY WARNING SIMULATION
# ========================================
print("\n" + "=" * 60)
print("EARLY WARNING SCENARIO")
print("=" * 60)
# Goal: Predict failure at cycle 15 (5 cycles early)
PREDICTION_CYCLE = 15
# Create early prediction dataset (only first 15 cycles)
X_test_early = X_test[:, :PREDICTION_CYCLE, :]
X_test_early_tensor = torch.FloatTensor(X_test_early)
print(f"\nEarly prediction setup:")
print(f"  Full sequence:     {SEQ_LENGTH} cycles")
print(f"  Prediction point:  Cycle {PREDICTION_CYCLE}")
print(f"  Early warning:     {SEQ_LENGTH - PREDICTION_CYCLE} cycles ahead")
print(f"  Test samples:      {X_test_early.shape[0]:,}")
# ========================================
# SUMMARY STATISTICS
# ========================================
print("\n" + "=" * 60)
print("DATASET SUMMARY")
print("=" * 60)
summary_df = pd.DataFrame({
    'Split': ['Train', 'Validation', 'Test', 'Total'],
    'Total': [len(y_train), len(y_val), len(y_test), NUM_DEVICES],
    'Pass': [(y_train==0).sum(), (y_val==0).sum(), (y_test==0).sum(), (y==0).sum()],
    'Fail': [(y_train==1).sum(), (y_val==1).sum(), (y_test==1).sum(), (y==1).sum()],
    'Fail %': [
        f"{(y_train==1).sum()/len(y_train)*100:.1f}%",
        f"{(y_val==1).sum()/len(y_val)*100:.1f}%",
        f"{(y_test==1).sum()/len(y_test)*100:.1f}%",
        f"{(y==1).sum()/NUM_DEVICES*100:.1f}%"
    ]
})
print("\n", summary_df.to_string(index=False))
print("\n" + "=" * 60)
print("✓ Data preparation complete!")
print("=" * 60)
# Save data for later use
print("\nSaving processed data...")
torch.save({
    'X_train': X_train_tensor,
    'y_train': y_train_tensor,
    'X_val': X_val_tensor,
    'y_val': y_val_tensor,
    'X_test': X_test_tensor,
    'y_test': y_test_tensor,
    'X_test_early': X_test_early_tensor,
    'scaler': scaler,
    'feature_names': FEATURE_NAMES
}, 'sequential_test_data.pt')
print("✓ Data saved to sequential_test_data.pt")


# 🧠 Part 3: Model Implementations

## 📝 What We'll Build

We'll implement 4 architectures and compare them:
1. **Vanilla RNN** - Baseline (will struggle with 20-step sequences)
2. **LSTM** - Gold standard for long sequences
3. **GRU** - Efficient alternative (75% of LSTM parameters)
4. **Bidirectional LSTM** - Best accuracy (2× parameters)

**Training Strategy:**
- Optimizer: Adam (learning rate = 0.001)
- Loss: CrossEntropyLoss (binary classification)
- Epochs: 20
- Early stopping: Stop if validation loss doesn't improve for 5 epochs
- Metrics: Accuracy, Precision, Recall, F1-Score

**Model Architecture Pattern:**
```
Input (batch_size, seq_length=20, input_size=15)
    ↓
RNN/LSTM/GRU Layer (hidden_size=128)
    ↓
Take final hidden state h_20
    ↓
Fully Connected Layer (128 → 2)
    ↓
Softmax → [P(pass), P(fail)]
```

### 📝 Implementation

**Purpose:** Core implementation with detailed code

**Key implementation details below.**

In [ ]:
# ========================================
# PART 3: MODEL IMPLEMENTATIONS
# RNN, LSTM, GRU, Bidirectional LSTM
# ========================================
import torch
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# ========================================
# MODEL 1: VANILLA RNN
# ========================================
class VanillaRNN(nn.Module):
    """
    Simple RNN for sequence classification
    Expected to struggle with long sequences (vanishing gradient)
    """
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # RNN layer
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True  # Input shape: (batch, seq, features)
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate RNN
        # out: (batch, seq_length, hidden_size)
        # hn: (num_layers, batch, hidden_size)
        out, hn = self.rnn(x, h0)
        
        # Take output from last time step
        out = out[:, -1, :]  # (batch, hidden_size)
        
        # Pass through fully connected layer
        out = self.fc(out)  # (batch, num_classes)
        
        return out
# ========================================
# MODEL 2: LSTM
# ========================================
class LSTMModel(nn.Module):
    """
    LSTM for sequence classification
    Handles long-term dependencies via cell state
    """
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        # out: (batch, seq_length, hidden_size)
        # hn, cn: (num_layers, batch, hidden_size)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Take output from last time step
        out = out[:, -1, :]
        
        # Pass through fully connected layer
        out = self.fc(out)
        
        return out


### 📝 Implementation Part 2

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# MODEL 3: GRU
# ========================================
class GRUModel(nn.Module):
    """
    GRU for sequence classification
    Simplified gating mechanism (fewer parameters than LSTM)
    """
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # GRU layer
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate GRU
        out, hn = self.gru(x, h0)
        
        # Take output from last time step
        out = out[:, -1, :]
        
        # Pass through fully connected layer
        out = self.fc(out)
        
        return out
# ========================================
# MODEL 4: BIDIRECTIONAL LSTM
# ========================================
class BiLSTMModel(nn.Module):
    """
    Bidirectional LSTM for sequence classification
    Processes sequence in both directions (forward + backward)
    """
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True  # Key difference!
        )
        
        # Fully connected layer (input = 2*hidden_size because bidirectional)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        # Initialize hidden and cell states (2*num_layers for bidirectional)
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate bidirectional LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Take output from last time step
        # out[:, -1, :] contains [forward_hidden; backward_hidden]
        out = out[:, -1, :]
        
        # Pass through fully connected layer
        out = self.fc(out)
        
        return out


### 📝 Implementation Part 3

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# TRAINING FUNCTION
# ========================================
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, patience=5):
    """
    Train RNN/LSTM/GRU model with early stopping
    
    Args:
        model: PyTorch model
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Maximum epochs
        learning_rate: Learning rate for Adam optimizer
        patience: Early stopping patience (epochs without improvement)
    
    Returns:
        Dictionary with training history
    """
    
    # Move model to device
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }
    
    # Early stopping
    best_val_loss = float('inf')
    patience_counter = 0
    
    print(f"\nTraining {model.__class__.__name__}...")
    print("=" * 60)
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_correct / val_total
        
        # Save history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_acc'].append(train_accuracy)
        history['val_acc'].append(val_accuracy)
        
        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
            print(f"  Val Loss:   {avg_val_loss:.4f}, Val Acc:   {val_accuracy:.4f}")
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), f'best_{model.__class__.__name__}.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping at epoch {epoch+1} (patience={patience})")
                break
    
    training_time = time.time() - start_time
    
    print(f"\nTraining completed in {training_time:.2f} sec ({training_time/60:.2f} min)")
    print("=" * 60)
    
    # Load best model
    model.load_state_dict(torch.load(f'best_{model.__class__.__name__}.pth'))
    
    history['training_time'] = training_time
    history['best_val_loss'] = best_val_loss
    
    return history


### 📝 Implementation Part 4

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# EVALUATION FUNCTION
# ========================================
def evaluate_model(model, test_loader):
    """
    Evaluate model on test set
    
    Returns:
        Dictionary with metrics
    """
    model = model.to(device)
    model.eval()
    
    all_predictions = []
    all_labels = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    inference_time = time.time() - start_time
    
    # Compute metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_predictions, average='binary'
    )
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'inference_time': inference_time,
        'samples': len(all_labels)
    }
    
    return metrics
# ========================================
# INITIALIZE ALL MODELS
# ========================================
INPUT_SIZE = 15      # 15 parametric features
HIDDEN_SIZE = 128    # Hidden units
NUM_CLASSES = 2      # Binary classification (pass/fail)
NUM_LAYERS = 1       # Single layer RNN/LSTM/GRU
print("\n" + "=" * 60)
print("INITIALIZING MODELS")
print("=" * 60)
models = {
    'VanillaRNN': VanillaRNN(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS),
    'LSTM': LSTMModel(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS),
    'GRU': GRUModel(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS),
    'BiLSTM': BiLSTMModel(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS)
}
# Print parameter counts
for name, model in models.items():
    num_params = sum(p.numel() for p in model.parameters())
    print(f"\n{name}:")
    print(f"  Parameters: {num_params:,}")
    print(f"  Architecture: {model}")


### 📝 Implementation Part 5

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# TRAIN ALL MODELS
# ========================================
print("\n" + "=" * 60)
print("TRAINING ALL MODELS")
print("=" * 60)
training_histories = {}
for name, model in models.items():
    history = train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001)
    training_histories[name] = history
# ========================================
# EVALUATE ALL MODELS
# ========================================
print("\n" + "=" * 60)
print("EVALUATING ALL MODELS ON TEST SET")
print("=" * 60)
evaluation_results = {}
for name, model in models.items():
    print(f"\nEvaluating {name}...")
    metrics = evaluate_model(model, test_loader)
    evaluation_results[name] = metrics
    
    print(f"  Accuracy:  {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1-Score:  {metrics['f1_score']:.4f}")
    print(f"  Inference: {metrics['inference_time']:.2f} sec ({metrics['inference_time']/metrics['samples']*1000:.2f} ms/sample)")
# ========================================
# VISUALIZATION: TRAINING CURVES
# ========================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("Training Curves: All Models", fontsize=16, fontweight='bold')
colors = ['blue', 'red', 'green', 'orange']
# Loss curves
ax = axes[0, 0]
for (name, history), color in zip(training_histories.items(), colors):
    ax.plot(history['train_loss'], label=f'{name} Train', color=color, linestyle='-', alpha=0.7)
    ax.plot(history['val_loss'], label=f'{name} Val', color=color, linestyle='--', alpha=0.7)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_title("Training and Validation Loss")
ax.legend()
ax.grid(True, alpha=0.3)
# Accuracy curves
ax = axes[0, 1]
for (name, history), color in zip(training_histories.items(), colors):
    ax.plot(history['train_acc'], label=f'{name} Train', color=color, linestyle='-', alpha=0.7)
    ax.plot(history['val_acc'], label=f'{name} Val', color=color, linestyle='--', alpha=0.7)
ax.set_xlabel("Epoch")
ax.set_ylabel("Accuracy")
ax.set_title("Training and Validation Accuracy")
ax.legend()
ax.grid(True, alpha=0.3)
# Final test accuracy comparison
ax = axes[1, 0]
names = list(evaluation_results.keys())
accuracies = [evaluation_results[name]['accuracy'] for name in names]
bars = ax.bar(names, accuracies, color=colors, alpha=0.7)
ax.set_ylabel("Accuracy")
ax.set_title("Test Set Accuracy Comparison")
ax.set_ylim(0.7, 1.0)
ax.grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
# Inference speed comparison
ax = axes[1, 1]
names = list(evaluation_results.keys())
speeds = [evaluation_results[name]['inference_time']/evaluation_results[name]['samples']*1000 
          for name in names]
bars = ax.bar(names, speeds, color=colors, alpha=0.7)
ax.set_ylabel("Time (ms/sample)")
ax.set_title("Inference Speed Comparison")
ax.grid(True, alpha=0.3, axis='y')
# Add value labels
for bar, speed in zip(bars, speeds):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{speed:.2f}', ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.savefig('rnn_lstm_gru_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Training curves saved to rnn_lstm_gru_comparison.png")


### 📝 Implementation Part 6

**Purpose:** Continue implementation

**Key implementation details below.**

In [ ]:
# ========================================
# CONFUSION MATRICES
# ========================================
fig, axes = plt.subplots(1, 4, figsize=(20, 4))
fig.suptitle("Confusion Matrices: All Models", fontsize=16, fontweight='bold')
for ax, (name, metrics) in zip(axes, evaluation_results.items()):
    cm = metrics['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
                xticklabels=['Pass', 'Fail'], yticklabels=['Pass', 'Fail'])
    ax.set_title(f"{name}\nAcc={metrics['accuracy']:.3f}")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Confusion matrices saved to confusion_matrices.png")
print("\n" + "=" * 60)
print("✓ All models trained and evaluated!")
print("=" * 60)


# 🚀 Part 4: Real-World Projects & Best Practices

## 🔬 Semiconductor Projects (Post-Silicon Validation)

### **Project 1: Adaptive Test Sequence Optimization with LSTM**

**Objective:** Dynamically optimize test sequence order based on real-time parametric trends

**Business Value:** $30M-$100M/year from 40% test time reduction + early failure detection

**Architecture:**
```
Sequential Test Data (20 cycles × 15 params)
    ↓
Bidirectional LSTM (256 hidden units, 2 layers)
    ↓
Attention Mechanism (weight important timesteps)
    ↓
Multi-task Output:
    ├─ Failure prediction (classification)
    ├─ Time-to-failure estimation (regression)
    └─ Next optimal test recommendation (ranking)
```

**Implementation Strategy:**
```python
class AdaptiveTestOptimizer(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Bidirectional LSTM encoder
        self.lstm = nn.LSTM(
            input_size=15, hidden_size=256, num_layers=2,
            batch_first=True, bidirectional=True
        )
        
        # Attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=512, num_heads=8
        )
        
        # Multi-task heads
        self.failure_classifier = nn.Linear(512, 2)  # Pass/Fail
        self.ttf_regressor = nn.Linear(512, 1)       # Time-to-failure
        self.test_ranker = nn.Linear(512, 20)        # Next test priority
    
    def forward(self, x):
        # Encode sequence
        lstm_out, _ = self.lstm(x)
        
        # Apply attention
        attn_out, attn_weights = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Global pooling
        pooled = attn_out.mean(dim=1)
        
        # Multi-task predictions
        failure_pred = self.failure_classifier(pooled)
        ttf_pred = self.ttf_regressor(pooled)
        test_ranking = self.test_ranker(pooled)
        
        return failure_pred, ttf_pred, test_ranking, attn_weights

# Training with multi-task loss
def multi_task_loss(failure_pred, ttf_pred, test_ranking, 
                    failure_label, ttf_label, test_order_label):
    # Classification loss
    loss_class = nn.CrossEntropyLoss()(failure_pred, failure_label)
    
    # Regression loss (only for failing devices)
    mask = (failure_label == 1).float()
    loss_reg = (nn.MSELoss(reduction='none')(ttf_pred.squeeze(), ttf_label) * mask).mean()
    
    # Ranking loss (learn optimal test order)
    loss_rank = nn.BCEWithLogitsLoss()(test_ranking, test_order_label)
    
    # Combined loss
    total_loss = loss_class + 0.5 * loss_reg + 0.3 * loss_rank
    return total_loss

# Deploy in production
def adaptive_test_flow(device_data):
    """
    Real-time test optimization during production
    
    Algorithm:
    1. Run first 5 tests (baseline measurements)
    2. LSTM predicts failure probability
    3. If P(failure) > 0.8: Skip remaining tests → Bin as fail (save 75% test time)
    4. If P(failure) < 0.2: Run only critical tests (save 50% time)
    5. If 0.2 ≤ P(failure) ≤ 0.8: Run full test suite
    6. Update model weekly with new data (active learning)
    """
    
    # Run initial tests
    initial_data = run_tests(device_data, test_indices=[0, 1, 2, 3, 4])
    
    # Predict failure
    failure_prob, ttf, test_priority, _ = model(initial_data)
    
    if failure_prob[1] > 0.8:  # High confidence of failure
        decision = "FAIL"
        remaining_tests = []  # Skip all
        saved_time = 0.75
    elif failure_prob[0] > 0.8:  # High confidence of pass
        # Run only top-5 critical tests (ranked by model)
        critical_tests = test_priority.argsort(descending=True)[:5]
        remaining_tests = critical_tests.tolist()
        saved_time = 0.50
    else:  # Uncertain → Run full suite
        remaining_tests = list(range(5, 20))
        saved_time = 0.0
    
    # Execute remaining tests
    final_data = run_tests(device_data, test_indices=remaining_tests)
    
    return decision, saved_time
```

**Success Metrics:**
- **Test time reduction:** 40% average (60% for clear failures, 20% for passes)
- **Early detection accuracy:** ≥95% (5 cycles ahead)
- **False skip rate:** <2% (avoid missing marginal failures)
- **ROI:** $30M-$100M/year from faster test throughput

---

### **Project 2: Wafer-Level Yield Forecasting with GRU**

**Objective:** Predict daily wafer yield based on 30-day process parameter time series

**Business Value:** $50M-$200M/year from proactive process adjustments

**Data:**
- 30-day rolling window of wafer fabrication parameters
- 50 process variables (temperature, pressure, gas flow, etch rate, etc.)
- Daily wafer yield (% passing devices)

**Architecture:**
```python
class YieldForecaster(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Multi-layer GRU (faster than LSTM, sufficient for 30-day sequences)
        self.gru = nn.GRU(
            input_size=50,    # 50 process variables
            hidden_size=256,
            num_layers=3,
            batch_first=True,
            dropout=0.2
        )
        
        # Forecast head (predict next 7 days)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 7)  # 7-day forecast
    
    def forward(self, x):
        # x: (batch, 30 days, 50 features)
        gru_out, _ = self.gru(x)
        
        # Take last hidden state
        last_hidden = gru_out[:, -1, :]
        
        # Forecast
        x = torch.relu(self.fc1(last_hidden))
        forecast = self.fc2(x)  # (batch, 7) - 7-day yield forecast
        
        return forecast

# Anomaly detection (flag unusual patterns)
def detect_yield_anomalies(forecast, historical_yield):
    """
    Alert if forecasted yield deviates significantly from historical baseline
    """
    mean_yield = historical_yield.mean()
    std_yield = historical_yield.std()
    
    # Z-score for each forecast day
    z_scores = (forecast - mean_yield) / std_yield
    
    # Flag if |z| > 2 (beyond 2 standard deviations)
    anomalies = (z_scores.abs() > 2).any(dim=1)
    
    return anomalies, z_scores

# Production deployment
def daily_yield_monitoring():
    """
    Run every morning to forecast yield and trigger alerts
    """
    # Fetch last 30 days of process data
    process_data = fetch_fab_data(days=30)
    
    # Preprocess
    X = preprocess_process_data(process_data)
    
    # Forecast next 7 days
    forecast = model(X)
    
    # Check for anomalies
    historical = fetch_historical_yield(days=365)
    anomalies, z_scores = detect_yield_anomalies(forecast, historical)
    
    if anomalies.any():
        # Alert fab engineers
        send_alert(
            subject="Yield Anomaly Detected",
            message=f"Forecasted yield drop: {forecast.min():.2f}% (z={z_scores.min():.2f})",
            recipients=["fab_engineer@company.com", "process_manager@company.com"]
        )
        
        # Recommend root cause investigation
        feature_importance = compute_shap_values(model, X)
        top_features = feature_importance.argsort(descending=True)[:5]
        
        print("Investigate these process parameters:")
        for i, feat_idx in enumerate(top_features):
            print(f"  {i+1}. {FEATURE_NAMES[feat_idx]}: {feature_importance[feat_idx]:.3f}")
    
    return forecast
```

**Success Metrics:**
- **Forecast accuracy:** RMSE < 2% (absolute yield percentage)
- **Anomaly detection:** 90% recall (catch yield drops), 85% precision (minimize false alarms)
- **Lead time:** 7-day advance warning (vs 1-day with reactive monitoring)
- **Cost savings:** $50M-$200M/year from proactive process tuning

---

### **Project 3: Device Lifetime Prediction from Reliability Test Sequences**

**Objective:** Predict device lifetime (MTTF) from accelerated stress test time series

**Business Value:** $10M-$40M/year from improved reliability binning + reduced RMA costs

**Architecture:** LSTM with attention (focus on degradation inflection points)

---

### **Project 4: Multi-Fab Process Drift Detection**

**Objective:** Detect process drift across 5 fabs using synchronized parametric time series

**Architecture:** Bidirectional GRU with contrastive learning (learn fab-invariant features)

---

## 🌐 General AI/ML Projects

### **Project 5: Stock Price Prediction with LSTM**

**Objective:** Forecast next 7-day stock prices from 60-day historical data

**Architecture:** Stacked LSTM (3 layers, 256 hidden units) + Dropout (0.3)

**Challenges:** Non-stationary data (use differencing), high noise (ensemble 5 models)

---

### **Project 6: Natural Language Generation (Text Completion)**

**Objective:** Character-level language model (predict next character given sequence)

**Architecture:** GRU (512 hidden, 2 layers) trained on Wikipedia text corpus

**Application:** Auto-complete, code generation, creative writing

---

### **Project 7: Video Activity Recognition**

**Objective:** Classify human actions from video sequences (30 FPS, 5-sec clips)

**Architecture:** 
- CNN (ResNet-50) extracts spatial features from each frame
- LSTM (256 hidden) models temporal dynamics across frames
- Final FC layer classifies 101 action classes

**Dataset:** UCF-101 (13K videos, 101 actions)

---

### **Project 8: Music Generation with LSTM**

**Objective:** Generate piano music conditioned on genre/mood

**Architecture:** Bidirectional LSTM (512 hidden, 3 layers) trained on MIDI sequences

**Output:** Note-by-note music generation (pitch + duration + velocity)

---

## 🎓 Key Takeaways & Best Practices

### **Architecture Selection Guide**

| Sequence Length | Memory Needs | Parameters | Recommendation | Rationale |
|-----------------|--------------|------------|----------------|-----------|
| **Short (≤10 steps)** | Low | Any | Vanilla RNN or GRU | Vanishing gradient not critical |
| **Medium (10-50)** | Medium | Budget-conscious | **GRU** | 75% of LSTM params, 90% accuracy |
| **Medium (10-50)** | Medium | Best accuracy | **LSTM** | Gold standard, proven |
| **Long (50-200)** | High | Any | **LSTM or Transformer** | LSTM cell state prevents vanishing |
| **Very long (>200)** | High | Any | **Transformer** | Self-attention, no sequential bottleneck |
| **Bidirectional OK** | Any | 2× budget | **BiLSTM/BiGRU** | Best accuracy (+2-5% over unidirectional) |

---

### **Training Best Practices**

**1. Gradient Clipping** (prevent exploding gradients):
```python
# Clip gradients to max norm = 1.0
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
```

**2. Layer Normalization** (stabilize training):
```python
class LSTMWithLayerNorm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)
    
    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        normalized_out = self.layer_norm(lstm_out)
        return normalized_out, (hn, cn)
```

**3. Learning Rate Scheduling** (reduce LR when validation loss plateaus):
```python
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

# After each epoch
scheduler.step(val_loss)
```

**4. Dropout for Regularization**:
```python
self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, dropout=0.2)
```

**5. Batch Size Tuning**:
- Small batches (16-32): Better generalization, slower training
- Large batches (128-256): Faster training, may overfit
- **Recommendation:** Start with 64, tune based on validation loss

---

### **Common Pitfalls & Solutions**

| Problem | Symptom | Solution |
|---------|---------|----------|
| **Vanishing gradient** | Training stalls, loss doesn't decrease | Use LSTM/GRU, gradient clipping, reduce sequence length |
| **Exploding gradient** | NaN loss after few iterations | Gradient clipping (max_norm=1.0), lower LR |
| **Overfitting** | Train acc=95%, Val acc=70% | Dropout (0.2-0.5), L2 regularization, more data |
| **Slow convergence** | Loss decreases slowly | Increase LR (0.001 → 0.01), use Adam optimizer, batch normalization |
| **Mode collapse (generation)** | Model outputs repetitive text | Temperature sampling, nucleus sampling, beam search |
| **Memory errors** | CUDA OOM | Reduce batch size, use gradient checkpointing, smaller hidden size |

---

### **Production Deployment Checklist**

✅ **Model Optimization:**
- [ ] Convert to ONNX for framework independence
- [ ] Quantize to INT8 (3-4× smaller, 2× faster, <1% accuracy loss)
- [ ] Prune redundant connections (remove 30-50% weights)
- [ ] Use optimized LSTM implementations (cuDNN, MKL-DNN)

✅ **Inference Optimization:**
- [ ] Batch multiple sequences together (process 32-64 at once)
- [ ] Use stateful LSTMs (preserve hidden state across batches for streaming)
- [ ] Cache hidden states for sequential predictions
- [ ] Compile model with TorchScript/TensorRT

✅ **Monitoring & Maintenance:**
- [ ] Log prediction confidence distributions (detect distribution drift)
- [ ] Track sequence lengths (ensure within training range)
- [ ] A/B test new model versions (gradual rollout)
- [ ] Active learning: flag low-confidence predictions for labeling

✅ **Edge Deployment:**
- [ ] Use GRU instead of LSTM (25% fewer parameters)
- [ ] Reduce hidden size (256 → 128, minimal accuracy loss)
- [ ] Quantize to FP16 or INT8
- [ ] Target <10 MB model size for mobile/IoT devices

---

### **Performance Optimization: Speed vs Accuracy**

| Configuration | Parameters | Speed (samples/sec) | Accuracy | Use Case |
|---------------|------------|---------------------|----------|----------|
| **GRU-64** | 15K | 1200 | 82% | Edge devices, real-time |
| **GRU-128** | 55K | 800 | 87% | Balanced (recommended) |
| **LSTM-128** | 73K | 600 | 88% | Standard production |
| **LSTM-256** | 289K | 200 | 90% | High-accuracy applications |
| **BiLSTM-256** | 578K | 100 | 92% | Best accuracy (offline) |

**Recommendation for production:**
- Start with **GRU-128** (good accuracy, fast)
- Upgrade to **LSTM-128** if accuracy critical (+1%)
- Use **BiLSTM** only for offline batch processing

---

## 📚 What's Next?

**Upcoming Notebooks:**
- **057: Sequence-to-Sequence Models** → Encoder-decoder architecture, attention mechanism
- **058: Transformers & Attention** → Self-attention, multi-head attention, BERT/GPT foundations
- **059: Time Series Forecasting** → ARIMA, Prophet, N-BEATS, Temporal Fusion Transformers
- **060: Generative RNNs** → Text generation, music generation, variational RNNs

---

## ✅ Learning Objectives Review

1. ✅ **Sequential Data Processing** - Temporal dependencies, memory mechanism
2. ✅ **RNN Architecture** - Hidden state recurrence, BPTT
3. ✅ **Vanishing Gradient** - Why vanilla RNNs fail, $(W_{hh})^{20} \rightarrow 0$
4. ✅ **LSTM Networks** - Cell state, forget/input/output gates, gradient flow
5. ✅ **GRU Networks** - Reset/update gates, 75% parameters of LSTM
6. ✅ **Bidirectional RNNs** - Forward + backward context, 2× parameters
7. ✅ **Semiconductor Applications** - Test sequence analysis, yield forecasting
8. ✅ **Production Deployment** - ONNX export, quantization, edge optimization

**Key Skill Acquired:** Build production-grade RNN/LSTM/GRU models for sequential data analysis!

---

## 📖 Additional Resources

**Must-Read Papers:**
- "Long Short-Term Memory" (Hochreiter & Schmidhuber, 1997) - Original LSTM paper
- "Learning Phrase Representations using RNN Encoder-Decoder" (Cho et al., 2014) - GRU introduction
- "Sequence to Sequence Learning with Neural Networks" (Sutskever et al., 2014) - Seq2seq architecture

**Courses & Tutorials:**
- CS224n (Stanford) - NLP with Deep Learning (RNN lectures)
- Fast.ai Deep Learning - RNN for text classification
- PyTorch LSTM Tutorial - https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

**Libraries & Tools:**
- **PyTorch** - https://pytorch.org/docs/stable/nn.html#recurrent-layers
- **TensorFlow** - https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
- **Hugging Face** - https://huggingface.co/transformers (Transformer-based models)

---

## 🎯 Final Summary

**RNN Family Mastery:**
- **Vanilla RNN:** Simple but struggles with long sequences (vanishing gradient)
- **LSTM:** Gold standard (cell state bypasses vanishing gradient)
- **GRU:** Efficient alternative (75% parameters, 90% accuracy of LSTM)
- **Bidirectional:** Best accuracy (forward + backward context)

**Semiconductor Impact:**
- **Test optimization:** $30M-$100M/year from adaptive test sequencing
- **Yield forecasting:** $50M-$200M/year from proactive process tuning
- **Reliability prediction:** $10M-$40M/year from improved lifetime estimation

**You're now ready to build sequential models for time-series analysis!** 🚀

---

**Congratulations on completing Notebook 056!** 🎉

Next notebook: **057_Seq2Seq_Attention.ipynb** - Encoder-decoder architectures for sequence transformation!